# Análise de Churn de Clientes – Telecom X

**Autor:** Seu Nome  
**Data:** 2025-06-XX  
**Projeto:** Challenge Telecom X  

**Resumo:**  
Este notebook consolida todo o processo de ETL, limpeza, análise exploratória e insights para reduzir o churn (evasão) de clientes na Telecom X.  


## 1. Introdução

A Telecom X tem observado um crescente índice de evasão de clientes (churn), impactando diretamente sua receita e capacidade de manter um fluxo recorrente estável.  
O objetivo desta análise é:

1. **Entender** o perfil dos clientes que cancelam o serviço em comparação com os que permanecem.
2. **Identificar** padrões em dados demográficos (gênero, idade, dependentes), comportamentais (tipo de plano, serviços contratados) e financeiros (valor gasto, tempo de contrato).
3. **Gerar insights** e **recomendações** que possam ser utilizados pela equipe de marketing e retenção para reduzir o churn.


In [ ]:
# 2. Importar bibliotecas necessárias
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)

# Definir caminhos (ajuste de acordo com a organização do seu projeto)
BASE_DIR    = os.getcwd()  # Em Jupyter, assume a pasta onde o notebook está salvo
CLEAN_CSV   = os.path.join(BASE_DIR, "data", "clean", "telecom_churn_cleaned.csv")
FEATURES_CSV = os.path.join(BASE_DIR, "data", "clean", "telecom_churn_features.csv")
TRANSFORMED_CSV = os.path.join(BASE_DIR, "data", "clean", "telecom_churn_transformed.csv")
REPORTS_DIR = os.path.join(BASE_DIR, "reports")

# Carregar os dados (usar o CSV final transformado para todas as análises)
df = pd.read_csv(TRANSFORMED_CSV)
print("Dimensões do DataFrame transformado:", df.shape)
df.head(3)
